In [31]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import os
import numpy as np
from datetime import datetime
import dateutil.relativedelta
import configparser

In [32]:
import requests
res = requests.get('https://telegram.org/')
print(res)

<Response [200]>


In [33]:
# %%capture
# %pip install -r requirements.txt
# !python -m nltk.downloader stopwords
# !python -m nltk.downloader wordnet
# !python -m nltk.downloader omw-1.4

In [35]:
if not os.path.exists('Last_Updated.txt'):
    last_updated_time = str(
        datetime.now() + dateutil.relativedelta.relativedelta(months=-1))
    CurrentYear,CurrentMonth,CurrentDate = [int(i) for i in last_updated_time.split(" ")[0].split("-")]
else:
    with open('Last_Updated.txt', 'r') as f:
        last_updated_time = f.read()
        CurrentYear,CurrentMonth,CurrentDate = [int(i) for i in last_updated_time.split(" ")[0].split("-")]

In [36]:
import ast
config = configparser.ConfigParser()
config.read_file(open('config.cfg'))
binance_coins= ast.literal_eval(config.get('Telegram', 'Coins'))
groups = ast.literal_eval(config.get('Telegram', 'Groups'))

In [37]:
import snscrape.modules.telegram as telegram
import pandas as pd

tweets_list = []
# for i in groups:
for x,tel in enumerate(telegram.TelegramChannelScraper("BitmexCourses").get_items()):
    y,m,d = [int(i) for i in str(tel.date).split(' ')[0].split('-')]
    if  (m<=CurrentMonth and d<=CurrentDate):
        break
    tweets_list.append(["BitmexCourses",str(tel.date).split(' ')[0],str(tel.date).split(' ')[1].split('+')[0], tel.content])
    

In [38]:
tel_df = pd.DataFrame(tweets_list, columns=['Name','Date','Time','Text'])

In [39]:
import re
def MoreCleaning(text):
    str_en = text.encode("ascii", "replace").replace(b"?",b" ")
    str_de = str_en.decode()
    str_de = re.sub(' +', ' ', re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", str_de).strip())
    str_de = re.sub(r"\bStop\sLoss\b", "StopLoss", str_de,re.IGNORECASE)
    print(str_de)
    return str_de
tel_df["Text"] = tel_df["Text"].apply(MoreCleaning)
tel_df["Text"][1]

#BNX #BNXUSDT (Binance)Buy between 0.57 - 0.62 Targets 0.67 - 0.72 - 0.77 - 0.86 StopLoss 0.52 https://www.tradingview.com/x/RtTacGFt/ @BitmexCourses
#BTC #BTCUSDT (Binance)ENTRY 28500 - 29800 Targets 27300 - 26600 - 25800 - 24500 StopLoss 31500 Leverage : 2 xhttps://www.tradingview.com/x/ 8 N 2 axplf/ @BitmexCourses
Crypto Bull Crypto Experts pinned a photo
Deposit $ARB to Bybit and Share Rewards From Our $ 400 , 000 $USDT Prize Pools! Learn how to claim and maximize your $ARB Airdrop with Bybit's Deposit and Staking campaigns: 1 Deposit 250 ARB to Earn 25 $USDT 2 Arbitrum Launchpool with 150 , 000 $ARB prize pool 3 [GMX Users Exclusive] Deposit 500 $ARB to Earn 0.5 $GMX 4 [Spot Grid Bot] Trade to Earn from the 20 , 000 $USDT Prize Pool! Also providing hands-on tutorial to claim your $ARB airdrop! Bybit will be the first exchange to list $ARB upon listing on March 23 , 1 PM UTC. Check details here
#ADA #ADAUSDT (Binance)Buy Between 0.360 - 0.387 Targets 0.40 - 0.43 - 0.48 - 0.58 StopL

'#BTC #BTCUSDT (Binance)ENTRY 28500 - 29800 Targets 27300 - 26600 - 25800 - 24500 StopLoss 31500 Leverage : 2 xhttps://www.tradingview.com/x/ 8 N 2 axplf/ @BitmexCourses'

In [40]:
tel_df= tel_df.iloc[::-1]


In [41]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
exclude.remove('.')
exclude.remove('-')
lemma = WordNetLemmatizer()
def text_cleaning(text):
    stop_free = ' '.join([word for word in str(text).split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

tel_df["Cleaned Text"]= tel_df["Text"].apply(text_cleaning)

In [42]:
print(tel_df["Cleaned Text"][4])

['ADA', 'ADAUSDT', 'BinanceBuy', 'Between', '0.360', '-', '0.387', 'Targets', '0.40', '-', '0.43', '-', '0.48', '-', '0.58', 'StopLoss', '0.30', 'httpswww.tradingview.comxqc', '4', 'W', '9', 'Iz', '1', 'BitmexCourses']


In [43]:
tel_df.groupby('Name').count()

,Date,Time,Text,Cleaned Text
Name,,,,
BitmexCourses,25,25,25,25


In [44]:
if not os.path.exists('Channels'):
    os.mkdir('Channels')
os.chdir(os.path.join(os.getcwd(),'Channels'))
for i in groups:
    if not os.path.exists(i):
        os.mkdir(i)
    os.chdir(os.path.join(os.getcwd(),i))
    tel_df[tel_df['Name']==i].to_csv('0_'+i+'.csv', mode='w', index=False, header=False)
    os.chdir('./Channels')
os.chdir("C:Users/Mohib/Desktop/FinalYear/Telegram")
with open('datetime.txt', 'w') as f:
    f.write(str(datetime.now()))

FileNotFoundError: [WinError 2] The system cannot find the file specified: './Channels'

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
count = []
names = []
for group in groups:
    names.append(group)
    count.append(len(tel_df[tel_df['Name']==group]))
plt.bar(names,count)
plt.xticks(rotation=90)
plt.show()

Before Filter 1

In [ ]:
def extractor(i):
    i = str(i)
    suffixes = 'usdt|busd|futures'
    coinsregex = '|'.join(binance_coins)
    found = False
    text = re.sub(r'[^\w\s]', ' ', i)
    for s in text.split():
        if re.search("({})({})+|({})+({})|^({})$|[0-9]+({})|({})[0-9]+".format(coinsregex,suffixes,suffixes,coinsregex,coinsregex,coinsregex,coinsregex), s,re.IGNORECASE):
            found = True
            break
    if(found):
        return i
    else: return np.nan
tel_df['Cleaned Text'] = tel_df['Cleaned Text'].apply(extractor)
tel_df_fil1 = tel_df[tel_df['Cleaned Text'].notna()] 

In [ ]:
tel_df_fil1

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
count = list()
names = list()
for group in groups:
    names.append(group)
    count.append(len(tel_df_fil1[tel_df_fil1['Name']==group]))
plt.bar(names,count)
plt.xticks(rotation=90)
plt.show()

After Filter 1 

In [ ]:
os.chdir("/workspace/FinalYear/")
if not os.path.exists('Channels'):
    os.mkdir('Channels')
os.chdir(os.path.join(os.getcwd(),'Channels'))
for i in groups:
    if not os.path.exists(i):
        os.mkdir(i)
    os.chdir(os.path.join(os.getcwd(),i))
    tel_df_fil1[tel_df_fil1['Name']==i].to_csv('1_'+i+'.csv', mode='a', index=False, header=False)
    os.chdir('/workspace/FinalYear/Channels')
os.chdir("/workspace/FinalYear/")
with open('datetime.txt', 'w') as f:
    f.write(str(datetime.now()))